# Lab 2: Pods và Containers

## 🎯 Objectives
- Hiểu Pods là gì và cách hoạt động
- Tạo và quản lý Pods
- Multi-container pods
- Pod lifecycle và debugging

## 📋 Prerequisites
- Hoàn thành Lab 1
- K8s cluster đang chạy


## 1. Pods là gì?

**Pod** là smallest deployable unit trong Kubernetes:
- Một hoặc nhiều containers chạy cùng nhau
- Share network và storage
- Cùng lifecycle (start/stop cùng nhau)
- Được schedule trên cùng một node


In [ ]:
# Setup để chạy lệnh qua WSL từ Windows
# Cell này sẽ tự động gọi kubectl thông qua WSL

import subprocess
import sys

# Kiểm tra xem đang chạy trên Windows hay không
IS_WINDOWS = sys.platform == 'win32'

def run_wsl(cmd):
    """Run command through WSL from Windows"""
    if IS_WINDOWS:
        # Chạy lệnh qua WSL
        full_cmd = f'wsl {cmd}'
    else:
        full_cmd = cmd
    
    try:
        result = subprocess.run(
            full_cmd,
            shell=True,
            capture_output=True,
            text=True,
            timeout=300
        )
        output = result.stdout + result.stderr
        return output if output.strip() else "No output"
    except subprocess.TimeoutExpired:
        return "Error: Command timed out"
    except Exception as e:
        return f"Error: {str(e)}"

print(f"🖥️ Running on: {'Windows (will use WSL)' if IS_WINDOWS else 'Linux/Mac'}")
print("\n🔍 Checking minikube status...")
print(run_wsl("minikube status"))

In [ ]:
# Start minikube nếu chưa chạy
# Chỉ cần chạy cell này nếu minikube chưa start

print("🚀 Starting minikube (có thể mất 1-2 phút)...")
print(run_wsl("minikube start"))
print("\n✅ Done! Kiểm tra lại status:")
print(run_wsl("minikube status"))

In [ ]:
import subprocess
import yaml

def run_kubectl(command):
    """Run kubectl command through WSL"""
    # Gọi kubectl thông qua WSL
    return run_wsl(command)

# Check kubectl version
print("🔍 Checking kubectl version:")
print(run_kubectl("kubectl version --client"))

# Create a simple pod
pod_manifest = """
apiVersion: v1
kind: Pod
metadata:
  name: simple-pod
  labels:
    app: simple-app
spec:
  containers:
  - name: app
    image: python:3.10-slim
    command: ["python", "-m", "http.server", "8000"]
    ports:
    - containerPort: 8000
"""

print("📝 Pod Manifest:")
print(pod_manifest)
print("\n💡 To create: kubectl apply -f manifests/01-pod-example.yaml")


In [ ]:
# Check pod status
print("📊 Pod Status:")
print(run_kubectl("kubectl get pods"))

print("\n📋 Detailed Pod Information:")
print(run_kubectl("kubectl describe pod simple-pod"))


## 2. Pod Lifecycle

Pod states:
- **Pending**: Đang được schedule
- **Running**: Pod đang chạy
- **Succeeded**: Containers completed successfully
- **Failed**: Containers failed
- **Unknown**: Không thể xác định state


In [ ]:
# View pod logs
print("📜 Pod Logs:")
print(run_kubectl("kubectl logs simple-pod"))

# Execute command in pod
print("\n🔧 Execute command in pod:")
print(run_kubectl("kubectl exec simple-pod -- ps aux"))


## 3. Multi-container Pods

Một Pod có thể chứa nhiều containers:
- Share network namespace
- Share storage volumes
- Cùng lifecycle
- Use cases: Sidecar pattern, logging, monitoring


In [ ]:
# Multi-container pod example
multi_pod = """
apiVersion: v1
kind: Pod
metadata:
  name: multi-container-pod
spec:
  containers:
  - name: app
    image: python:3.10-slim
    command: ["python", "-m", "http.server", "8000"]
  - name: sidecar
    image: busybox
    command: ["sh", "-c", "while true; do echo 'Sidecar running'; sleep 30; done"]
"""

print("📝 Multi-container Pod:")
print(multi_pod)
